In [2]:
from riotwatcher import LolWatcher
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv


load_dotenv()  # carga las variables del archivo .env
API_KEY = os.getenv("RIOT_API_KEY")  # ahora api_key tiene tu clave

# Configuración
RIOT_ID_NAME = "Stitch"
RIOT_ID_TAG = "FPMR"
REGION = "AMERICAS"  # Para cuentas LAS
QUEUE = 440  # 440 Flex Ranked o 420 para soloQ

# --- 1. Obtener PUUID del jugador ---
def get_puuid(api_key, riot_id_name, riot_id_tag):
    url = f"https://{REGION}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{riot_id_name}/{riot_id_tag}?api_key={api_key}"
    response = requests.get(url)

    print("Status code:", response.status_code)
    print("Respuesta JSON:", response.json())  # DEBUG

    if response.status_code == 200:
        return response.json().get("puuid")
    else:
        raise ValueError(f"Error al obtener PUUID: {response.status_code} - {response.json()}")

puuid = get_puuid(API_KEY, RIOT_ID_NAME, RIOT_ID_TAG)

# --- 2. Obtener 100 partidas Flex Ranked ---
watcher = LolWatcher(API_KEY)
matches = watcher.match.matchlist_by_puuid(REGION, puuid, queue=QUEUE, count=100)

# --- 3. Extraer TODAS las features posibles ---
def extract_all_features(match_data, puuid):
    participant = next(p for p in match_data["info"]["participants"] if p["puuid"] == puuid)
    
    features = {
        # Metadata
        "match_id": match_data["metadata"]["matchId"],
        "champion": participant["championName"],
        "win": int(participant["win"]),  # 1 o 0 para ML
        "queue_id": match_data["info"]["queueId"],
        "game_duration": match_data["info"]["gameDuration"] // 60,  # Minutos
        "patch": match_data["info"]["gameVersion"],
        
        # Estadísticas básicas
        "kills": participant["kills"],
        "deaths": participant["deaths"],
        "assists": participant["assists"],
        "kda": (participant["kills"] + participant["assists"]) / max(1, participant["deaths"]),
        "role": participant["teamPosition"],
        "lane": participant["lane"],
        
        # Oro y farm
        "gold_earned": participant["goldEarned"],
        "gold_spent": participant["goldSpent"],
        "total_minions": participant["totalMinionsKilled"],
        "neutral_minions": participant["neutralMinionsKilled"],
        
        # Daño
        "total_damage": participant["totalDamageDealtToChampions"],
        "physical_damage": participant["physicalDamageDealtToChampions"],
        "magic_damage": participant["magicDamageDealtToChampions"],
        "true_damage": participant["trueDamageDealtToChampions"],
        "damage_taken": participant["totalDamageTaken"],
        "damage_healed": participant["totalHeal"],
        
        # Visión
        "vision_score": participant["visionScore"],
        "wards_placed": participant["wardsPlaced"],
        "wards_killed": participant["wardsKilled"],
        "vision_wards": participant["visionWardsBoughtInGame"],
        
        # Objetivos
        "turret_kills": participant["turretKills"],
        "inhibitor_kills": participant["inhibitorKills"],
        "dragon_kills": participant["dragonKills"],
        "baron_kills": participant["baronKills"],
        "rift_herald_kills": participant.get("riftHeraldKills", 0),
        
        # Objetos (item0 a item6)
        **{f"item_{i}": participant[f"item{i}"] for i in range(7)},
        
        # Runas
        "primary_rune_tree": participant["perks"]["styles"][0]["style"],
        "secondary_rune_tree": participant["perks"]["styles"][1]["style"],
    }
    
    # Runas específicas (selecciones)
    for i, style in enumerate(participant["perks"]["styles"]):
        for j, selection in enumerate(style["selections"]):
            features[f"rune_{i}_{j}"] = selection["perk"]
    
    return features

# --- 4. Procesar partidas y guardar CSV ---
all_features = []
for i, match_id in enumerate(matches):
    try:
        match_data = watcher.match.by_id(REGION, match_id)
        features = extract_all_features(match_data, puuid)
        all_features.append(features)
        
        print(f"Procesada partida {i+1}/100 - ID: {match_id}")
        time.sleep(1.2)  # Respeta rate limits
        
    except Exception as e:
        print(f"Error en {match_id}: {e}")

# Crear DataFrame y guardar
df = pd.DataFrame(all_features)
df.to_csv("partidas_flex.csv", index=False)

print(f"\n✅ CSV generado con {len(df)} partidas y {len(df.columns)} features")
print("📊 Features disponibles:", df.columns.tolist())

Status code: 200
Respuesta JSON: {'puuid': 'B05Qvn3L1ayxo9Sgn0qNW3s9ZLQ3X-N82E1OSUt0aWNfUbFzU_ojLBJ8ljxMxRIhirhO4x1aRPViTA', 'gameName': 'Stitch', 'tagLine': 'FPMR'}
Procesada partida 1/100 - ID: LA2_1527080479
Procesada partida 2/100 - ID: LA2_1527065732
Procesada partida 3/100 - ID: LA2_1527057376
Procesada partida 4/100 - ID: LA2_1527048075
Procesada partida 5/100 - ID: LA2_1527038523
Procesada partida 6/100 - ID: LA2_1527031869
Procesada partida 7/100 - ID: LA2_1527025303
Procesada partida 8/100 - ID: LA2_1527016979
Procesada partida 9/100 - ID: LA2_1527015279
Procesada partida 10/100 - ID: LA2_1527006020
Procesada partida 11/100 - ID: LA2_1526997759
Procesada partida 12/100 - ID: LA2_1526993232
Procesada partida 13/100 - ID: LA2_1526968780
Procesada partida 14/100 - ID: LA2_1526969157
Procesada partida 15/100 - ID: LA2_1526959574
Procesada partida 16/100 - ID: LA2_1526952102
Procesada partida 17/100 - ID: LA2_1526939589
Procesada partida 18/100 - ID: LA2_1526926743
Procesada parti